In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import tensorflow as tf  # type: ignore # building and training deep learning model
import numpy as np
from tensorflow import keras  # type: ignore # high level APIs
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # type: ignore # Augmentation (note: preprocessing, not processing)
from tensorflow.keras.models import Sequential  # type: ignore # linear stack of neural network layers (note: Sequential, not sequential)
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout  # type: ignore # layer CNN (note: Flatten, not Flattern)
from tensorflow.keras.optimizers import Adam  # type: ignore # Optimizer for training (note: optimizers, not optimisers)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping  # type: ignore # training callbacks
import matplotlib.pyplot as plt

In [2]:
DATASET_DIR='flowers'

**Augment and preprocess the images**

In [3]:
#training images

train_datagen=ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images
    shear_range=0.2,  # Randomly shear images
    horizontal_flip=True,  # Randomly flip images horizontally
    zoom_range=0.2 , # Randomly zoom images
)


train_generator=train_datagen.flow_from_directory(
    os.path.join(DATASET_DIR,'train'),
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    )

Found 4317 images belonging to 5 classes.


In [4]:
#validation images
# Note: validation_datagen is used for validation data, not training data.

validation_datagen=ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
)

validation_generator=validation_datagen.flow_from_directory(
    os.path.join(DATASET_DIR,'validation'),
    target_size=(150,150),
    batch_size=32,
    class_mode='categorical',
    )

Found 4317 images belonging to 5 classes.


**Build our model**

In [5]:
model=Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)))  # Input shape for RGB images of size 150x150
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128,(3,3),activation='relu'))#increase to 128 the filter
model.add(MaxPooling2D(2,2))
model.add(Conv2D(256,(3,3),activation='relu'))#increase to 256 the filter 
model.add(MaxPooling2D(2,2))
model.add(Flatten())#flatten features map to iD vector
model.add(Dense(256,activation='relu'))#fully connected layer
model.add(Dropout(0.5))# drop to 50% of the neurons to prevent overfitting
model.add(Dense(512,activation='relu'))
model.add(Dense(5,activation='softmax'))
model.compile(optimizer=Adam(learning_rate=0.0001),
                   loss='categorical_crossentropy',
                  metrics=['accuracy'])

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
earlystopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)
checkpoint=ModelCheckpoint(filepath='flowers_model.h5',monitor='val_loss',save_best_only=True)

In [7]:
#train the model
history=model.fit(train_generator,epochs=15,validation_data=validation_generator,
                  callbacks=[earlystopping, checkpoint])

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3064 - loss: 1.5260

135/135 ━━━━━━━━━━━━━━━━━━━━ 221s 2s/step - accuracy: 0.3068 - loss: 1.5253 - val_accuracy: 0.4545 - val_loss: 1.2168
Epoch 2/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4810 - loss: 1.1993

135/135 ━━━━━━━━━━━━━━━━━━━━ 204s 2s/step - accuracy: 0.4811 - loss: 1.1992 - val_accuracy: 0.5603 - val_loss: 1.0783
Epoch 3/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5370 - loss: 1.1079

135/135 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.5371 - loss: 1.1078 - val_accuracy: 0.5608 - val_loss: 1.0644
Epoch 4/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5804 - loss: 1.0416

...

135/135 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.6210 - loss: 0.9579 - val_accuracy: 0.6644 - val_loss: 0.8730
Epoch 7/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6484 - loss: 0.9087

135/135 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.6483 - loss: 0.9087 - val_accuracy: 0.6965 - val_loss: 0.8013
Epoch 8/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6728 - loss: 0.8715

135/135 ━━━━━━━━━━━━━━━━━━━━ 227s 2s/step - accuracy: 0.6728 - loss: 0.8713 - val_accuracy: 0.6991 - val_loss: 0.7822
Epoch 9/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6644 - loss: 0.8551

135/135 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.6644 - loss: 0.8551 - val_accuracy: 0.7176 - val_loss: 0.7469
Epoch 10/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.6893 - loss: 0.8036 - val_accuracy: 0.6773 - val_loss: 0.8736
Epoch 11/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6868 - loss: 0.7968

135/135 ━━━━━━━━━━━━━━━━━━━━ 257s 2s/step - accuracy: 0.6869 - loss: 0.7967 - val_accuracy: 0.7382 - val_loss: 0.6955
Epoch 12/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 232s 2s/step - accuracy: 0.7036 - loss: 0.7835 - val_accuracy: 0.7169 - val_loss: 0.7459
Epoch 13/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.7253 - loss: 0.7350 - val_accuracy: 0.7223 - val_loss: 0.7412
Epoch 14/15
135/135 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.7243 - loss: 0.7466 - val_accuracy: 0.7204 - val_loss: 0.7159


In [11]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.savefig('training_history.png')
plt.close()

In [12]:
!pip install opencv-python

In [20]:
import cv2
import numpy as np

image_path = './flowers/train/daisy/10172379554_b296050f82_n.jpg'
flower_image = cv2.imread(image_path)

if flower_image is None:
    print(f"Error: Could not load image from path: {image_path}")
else:
    flower_image = cv2.resize(flower_image, (150, 150)) / 255.0
    flower_image = np.expand_dims(flower_image, axis=0)
    predictions = model.predict(flower_image)
    predicted_label = np.argmax(predictions[0])
    print(f"Predicted label: {predicted_label}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 292ms/step
Predicted label: 2


In [21]:
#map to actual classes
class_indices=train_generator.class_indices

predicted_class=[key for key,value in class_indices.items()if value==predicted_label][0]
print(f"Predicted class: {predicted_class}")

Predicted class: rose
